In [2]:
from ultralytics import YOLO
import cv2
import math


# Cambiar el índice de la cámara
cap = cv2.VideoCapture(1)  # Prueba con 1, 2, etc. si tienes varias cámaras conectadas

if not cap.isOpened():
    print("Error: No se pudo abrir la cámara")
else:
    print("Cámara abierta con éxito")
cap.set(3, 640)  # Ancho del video
cap.set(4, 480)  # Altura del video

# Cargar tu modelo personalizado
model = YOLO(r"models\best.pt")  # Ruta del modelo entrenado

# Clases personalizadas
classNames = ['greenBox', 'redBox', 'yellowBox']  # Clases de tu modelo

# Bucle para capturar frames de la webcam
while True:
    success, img = cap.read()
    if not success:
        print("Error al acceder a la cámara.")
        break

    # Realizar la predicción con el modelo
    results = model(img, stream=True)

    # Procesar las coordenadas de los objetos detectados
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Obtener las coordenadas del bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertir a valores enteros

            # Dibujar el bounding box en el frame
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Obtener la confianza
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confianza:", confidence)

            # Obtener la clase
            cls = int(box.cls[0])  # Índice de la clase
            class_name = classNames[cls] if cls < len(classNames) else "Unknown"
            print("Clase detectada:", class_name)

            # Agregar texto con el nombre de la clase y confianza
            org = (x1, y1 - 10)  # Posición del texto
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.5
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, f"{class_name} {confidence:.2f}", org, font, fontScale, color, thickness)

    # Mostrar el frame en la ventana
    cv2.imshow('Webcam', img)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


Cámara abierta con éxito

0: 480x640 (no detections), 180.4ms
Speed: 0.0ms preprocess, 180.4ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 108.8ms
Speed: 4.0ms preprocess, 108.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 93.0ms
Speed: 0.0ms preprocess, 93.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 75.5ms
Speed: 2.0ms preprocess, 75.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 77.5ms
Speed: 0.0ms preprocess, 77.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 92.5ms
Speed: 2.5ms preprocess, 92.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 91.4ms
Speed: 0.0ms preprocess, 91.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 73.0ms
Spee

In [14]:
from ultralytics import YOLO
import cv2
import math
import numpy as np


ZONE = np.array([[240, 165],[240, 380],[450, 380],[450, 165]])
# Ruta del video local
video_path = r"C:\Users\Miguel\Downloads\WIN_20241220_15_50_29_Pro.mp4"  # Cambia esta ruta por la de tu video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: No se pudo abrir el archivo de video")
    exit()

# Cargar tu modelo personalizado
model = YOLO(r"C:\Users\Miguel\Downloads\best.pt")  # Ruta del modelo entrenado
model.overrides['imgsz'] = 320  # Cambiar el tamaño de la entrada del modelo

# Clases personalizadas
classNames = ['greenBox', 'redBox', 'yellowBox']  # Clases de tu modelo

# Definir el tamaño deseado (más pequeño)
resize_width = 720
resize_height = 480

# Reducir la frecuencia de procesamiento
frame_skip = 5  # Procesar uno de cada 5 frames
frame_count = 0

# Contador de cajas que están dentro del polígono
counter = 0

# Función para comprobar si un punto está dentro del polígono
def is_point_in_polygon(point, polygon):
    return cv2.pointPolygonTest(polygon, tuple(point), False) >= 0

# Bucle para procesar frames del video
while True:
    success, img = cap.read()
    if not success:
        print("Fin del video o error al leer el frame.")
        break

    # Redimensionar el frame para adaptarlo al tamaño deseado
    img_resized = cv2.resize(img, (resize_width, resize_height), interpolation=cv2.INTER_AREA)

    # Incrementar el contador de frames
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Saltar frames no deseados

    # Dibujar el polígono en el frame
    cv2.polylines(img_resized, [ZONE], isClosed=True, color=(0, 255, 0), thickness=2)

    # Realizar la predicción con el modelo
    results = model(img_resized, stream=True)

    # Contador de cajas dentro del polígono
    count_inside_polygon = 0

    # Procesar las coordenadas de los objetos detectados
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Obtener las coordenadas del bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Calcular el centro de la caja
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            # Verificar si el centro de la caja está dentro del polígono
            if is_point_in_polygon([center_x, center_y], ZONE):
                count_inside_polygon += 1

            # Dibujar el bounding box en el frame
            cv2.rectangle(img_resized, (x1, y1), (x2, y2), (255, 0, 255), 2)

            # Obtener la confianza
            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])  # Índice de la clase
            class_name = classNames[cls] if cls < len(classNames) else "Unknown"

            # Agregar texto con el nombre de la clase y confianza
            cv2.putText(img_resized, f"{class_name} {confidence:.2f}",
                        (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Mostrar el contador de cajas dentro del polígono
    cv2.putText(img_resized, f"Cajas dentro del polígono: {count_inside_polygon}", (10, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Mostrar el frame redimensionado
    cv2.imshow('Video Detectado', img_resized)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


0: 224x320 (no detections), 38.7ms
Speed: 3.0ms preprocess, 38.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 43.5ms
Speed: 10.0ms preprocess, 43.5ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 42.0ms
Speed: 0.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 40.7ms
Speed: 11.0ms preprocess, 40.7ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 59.1ms
Speed: 0.0ms preprocess, 59.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 48.3ms
Speed: 0.0ms preprocess, 48.3ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 50.3ms
Speed: 0.0ms preprocess, 50.3ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 (no detections), 38.9ms
Speed: 0.0ms preprocess, 38.9ms

In [5]:
import cv2
from IPython.display import display, clear_output
import PIL.Image
import numpy as np

class Coordinates:
    def __init__(self, video_path: str):
        self.cap = cv2.VideoCapture(video_path)
        self.video_path = video_path

        # Crear ventana de eventos
        cv2.namedWindow("Frame")
        cv2.setMouseCallback("Frame", self.print_coordinates)

    def print_coordinates(self, event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            print(f"[{x}, {y}],")

    def video(self):
        try:
            while True:
                status, frame = self.cap.read()
                if not status:
                    print("Fin del video o error al leer el frame.")
                    break
                # Redimensionar el frame a 640x480
                frame = cv2.resize(frame, (640, 480))  # Ajusta el tamaño según tus necesidades

                # Mostrar el frame también con OpenCV para eventos del mouse
                cv2.imshow("Frame", frame)

                # Salir con 'q'
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        except KeyboardInterrupt:
            print("Ejecución interrumpida por el usuario.")
    
    def __del__(self):
        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    video_path = r"C:\Users\Miguel\Downloads\WIN_20241220_15_44_11_Pro.mp4"  # Cambia esta ruta al video que quieras
    c = Coordinates(video_path)
    c.video()

[365, 305],
[353, 466],
[401, 467],
[402, 327],


In [10]:
import os
from ultralytics import solutions
import cv2
import numpy as np
from ultralytics.utils.plotting import Annotator


os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Definir la región de interés (polígono)
ZONE = np.array([[240, 165], [240, 380], [450, 380], [450, 165]])

# Ruta del video local
video_path = r"C:\Users\Miguel\Downloads\WIN_20241220_15_50_29_Pro.mp4"  # Cambia esta ruta por la de tu video
output_path = r"C:\Users\Miguel\Downloads\video_detectado_output.avi"  # Archivo de salida

# Configuración de redimensionamiento del video
resize_width, resize_height = 720, 480  # Tamaño al que se redimensionará el video

# Abrir el video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: No se pudo abrir el archivo de video")
    exit()

# Obtener propiedades del video original
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Configurar el escritor de video
video_writer = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*'XVID'),
    fps,
    (resize_width, resize_height)
)

# Inicializar el ObjectCounter
counter = solutions.ObjectCounter(
    model=r"C:\Users\Miguel\Downloads\best.pt",  # Ruta del modelo YOLO entrenado
    region=ZONE,  # Región de interés
    show=False,  # Mostrar el video con detecciones
    classes=None,  # Si deseas contar todas las clases detectadas
    show_in=True,  # Mostrar objetos que entran al área
    show_out=False,  # Opcional: No mostrar los que salen
)

# Sobrescribir el método de dibujo de trayectorias

# Configuración para reducir los frames procesados
frame_skip = 5  # Saltar 4 frames entre cada uno procesado (por ejemplo, procesar solo 1 de cada 5 frames)
frame_count = 0  # Contador de frames

# Procesar el video
while True:
    success, frame = cap.read()
    if not success:
        print("Fin del video o error al leer el frame.")
        break

    # Redimensionar el frame al tamaño deseado
    frame_resized = cv2.resize(frame, (resize_width, resize_height), interpolation=cv2.INTER_AREA)

    # Incrementar el contador de frames
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Saltar el frame si no es el correcto
    

    processed_frame = counter.count(frame_resized)
    
    # Escribir el frame procesado en el archivo de salida
    video_writer.write(processed_frame)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Video Detectado', processed_frame)
    print(f"Inward count: {counter.in_count}, Outward count: {counter.out_count}")
    print(f"Muestra objetos por classe{counter.classwise_counts}")
    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Procesamiento completo. El video de salida se guardó en: {output_path}")




Ultralytics Solutions:  {'region': array([[240, 165],
       [240, 380],
       [450, 380],
       [450, 165]]), 'show_in': True, 'show_out': False, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'C:\\Users\\Miguel\\Downloads\\best.pt', 'show': False, 'classes': None}


AttributeError: 'NoneType' object has no attribute 'draw_centroid_and_tracks'

In [10]:
import os
from ultralytics import solutions
import cv2
import numpy as np

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Definir la región de interés (polígono)
ZONE = np.array([[400, 165], [400, 380], [450, 380], [450, 165]]) #240
#ZONE = np.array([[365, 300],[365, 450],[450, 450],[450, 300]])

# Ruta del video local
video_path = r"C:\Users\Miguel\Downloads\WIN_20241220_15_50_29_Pro.mp4"  # Cambia esta ruta por la de tu video
output_path = r"C:\Users\Miguel\Downloads\video_detectado_output.avi"  # Archivo de salida

# Configuración de redimensionamiento del video
resize_width, resize_height = 720, 480  # Tamaño al que se redimensionará el video

# Abrir el video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: No se pudo abrir el archivo de video")
    exit()

# Obtener propiedades del video original
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Configurar el escritor de video
video_writer = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*'XVID'),
    fps,
    (resize_width, resize_height)
)

# Inicializar el ObjectCounter sin mostrar anotaciones (show=False)
counter = solutions.ObjectCounter(
    model=r"C:\Users\Miguel\Downloads\best.pt",  # Ruta del modelo YOLO entrenado
    region=ZONE,  # Región de interés
    show=False,  # No mostrar el video con detecciones
    classes=None,  # Si deseas contar todas las clases detectadas
    show_in=True,  # Mostrar objetos que entran al área
    show_out=False,  # No mostrar los que salen
)


# Configuración para reducir los frames procesados
frame_skip = 5  # Saltar 4 frames entre cada uno procesado (por ejemplo, procesar solo 1 de cada 5 frames)
frame_count = 0  # Contador de frames

# Procesar el video
while True:
    success, frame = cap.read()
    if not success:
        print("Fin del video o error al leer el frame.")
        break

    # Redimensionar el frame al tamaño deseado
    frame_resized = cv2.resize(frame, (resize_width, resize_height), interpolation=cv2.INTER_AREA)

    # Incrementar el contador de frames
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Saltar el frame si no es el correcto

    # Procesar el frame con ObjectCounter
    processed_frame = counter.count(frame_resized)

    # Escribir el frame procesado en el archivo de salida
    video_writer.write(processed_frame)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Video Detectado', processed_frame)
    print(f"Inward count: {counter.in_count}, Outward count: {counter.out_count}")
    print(f"Muestra objetos por clase: {counter.classwise_counts}")

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Procesamiento completo. El video de salida se guardó en: {output_path}")

Ultralytics Solutions:  {'region': array([[400, 165],
       [400, 380],
       [450, 380],
       [450, 165]]), 'show_in': True, 'show_out': False, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'C:\\Users\\Miguel\\Downloads\\best.pt', 'show': False, 'classes': None}

0: 448x640 (no detections), 128.8ms
Speed: 23.0ms preprocess, 128.8ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)
WARNING  no tracks found!
Inward count: 0, Outward count: 0
Muestra objetos por clase: {}

0: 448x640 (no detections), 107.1ms
Speed: 3.9ms preprocess, 107.1ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
WARNING  no tracks found!
Inward count: 0, Outward count: 0
Muestra objetos por clase: {}

0: 448x640 (no detections), 85.6ms
Speed: 3.0ms preprocess, 85.6ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
WARNING  no tracks found!
Inward count: 0, Outward 

In [1]:
!python --version


Python 3.12.7
